In [ ]:
import os
# 你的LLM api 有溫度的用來生prompt才不會一直重複 沒溫度的用來評估prompt才不會不公平
from api.Mistral2_temperature_05_API import get_temperature_mistral
from api.Mistral2 import get_mistral

# 你的訓練資料集 我使用100筆 所以分數是0~100
training_data=[]

# 評估prompt的分數
def get_score(new_prompt,training_data):
   """
   將你練蠱時用來測試新prompt的data在這跑，之後回傳他的分數
   """
   score=get_mistral()
   return score

# 練蠱終止條件(我是設超過baseline做100題後的分數:60)
stop_score=60

# 起始prompt與分數(自己給幾個簡單prompt拿去get_score跑)
prompt_list=[
  {'prompt': "Let's think step by step.", 
   'score': 48},
  {'prompt': '', 
   'score': 46},
  {'prompt': ' "Identify the specific information that responds to the question in the article."',
   'score': 50}]

# 你要給LLM看幾個example
example_num=5

# 開練
while sorted_prompt_list[0]['score']<stop_score:
  # 單純用來終止迴圈 你也可以interrupt
  if os.path.exists(os.path.join(os.getcwd(),'stop_true.txt')):
      break
  
  # 排序你的prompt 拿比較高分的給LLM當example
  sorted_prompt_list = sorted(prompt_list, key=lambda x: x['score'], reverse=True)
  # 整理example格式
  example=""
  for p in sorted_prompt_list[:example_num]:
      example+=f"""[Old prompt]:"{p['prompt']}"
  [Scores]:{p['score']}

  """
  # 整理輸入(可以print出來檢查)
  input_to_temperature_llm=f"""You are an expert at crafting prompts.
  Based on the example task given below for an LLM, fill in the most suitable prompt in the place marked [new_prompt].
  The following describes the task you will undertake:

  "
  Article excerpt:
  [article_chunk]

  The above is the article excerpt related to my question.
  Below is the question I want to ask.
  Please select the text content that can answer this question.
  [new_prompt]

  Question:
  [input_question]
  "

  Here are some example prompts and their scores, ranging from 0 to 100, with higher scores indicating better performance.
  Please help me think of a unique new_prompt where higher scores are better.

  {example}

  ### You only need to return the new_prompt ###
  DON'T return the [Scores] or explanation.
  Your new_prompt:__"""
  new_prompt=get_temperature_mistral(input_to_temperature_llm)
  
  score=get_score(new_prompt,training_data)
  print("*"*50)
  print(f"{new_prompt}\n{score}")
  prompt_list.append({"prompt":new_prompt,
      "score":score})
